In [2]:
#Create input file for seismic tomography
#Source line : id_src year month day hour min sec lat lon dep_km magnitude num_recs id_event (weight)
#Receiver line : id_src id_rec name_rec lat lon elevation_m phase arrival_time_sec (weight)
import numpy as np
import pandas as pd
import SeisBlast as sb
from obspy import UTCDateTime
seisblast_files = '/home/bmelo/bin/SeisBlast/files/'

In [3]:
#Import all the necessary files
james_catalogue = pd.read_csv('/mnt/REPO/QUARRY-BLASTS/CATALOGS/quarry-blast-catalog-2013-2014.csv', sep = r'\s+')
james_bulletins_dir = "/mnt/REPO/QUARRY-BLASTS/BULLETINS/"
stations = pd.read_csv(
    "/home/bmelo/bin/Quarry/James_files/Total_Stations_List.txt", 
    delimiter=' ',
    header = 0,
    names=['sta', 'name','lat', 'lon', 'elevation', 'net', 'end'])
relocated_blasts = pd.read_csv('/home/bmelo/bin/SeisBlast/files/Eire_blasts_relocated.csv')
#merge with the relocated quarry blasts lists to get the relocated events
relocated_catalogue = pd.merge(james_catalogue, relocated_blasts, left_on='EVENT-ID', right_on='id-ev')


In [4]:

#start with formating the separate date and time formats
relocated_catalogue['year'] = pd.to_datetime(relocated_catalogue['DATE'], format='%Y-%m-%d').dt.year
relocated_catalogue['month'] = pd.to_datetime(relocated_catalogue['DATE'], format='%Y-%m-%d').dt.month
relocated_catalogue['day'] = pd.to_datetime(relocated_catalogue['DATE'], format='%Y-%m-%d').dt.day
relocated_catalogue['hour'] = pd.to_datetime(relocated_catalogue['TIME'], format='%H:%M:%S.%f').dt.hour
relocated_catalogue['minute'] = pd.to_datetime(relocated_catalogue['TIME'], format='%H:%M:%S.%f').dt.minute
relocated_catalogue['second'] = pd.to_datetime(relocated_catalogue['TIME'], format='%H:%M:%S.%f').dt.second + pd.to_datetime(relocated_catalogue['TIME'], format='%H:%M:%S.%f').dt.microsecond / 1e6
relocated_catalogue['depth-quarry'] = 0

In [5]:
#for running tests
index_src = 488
ev_id = relocated_catalogue['EVENT-ID'].iloc[index_src]
df, info = sb.list_picks(james_catalogue, james_bulletins_dir, ev_id)
origin_time = pd.to_datetime((relocated_catalogue.loc[index_src, 'DATE'] + ' ' + relocated_catalogue.loc[index_src, 'TIME']), format='%Y-%m-%d %H:%M:%S.%f')
df['Ptime(s)'] = (df['time']['P'] - origin_time).dt.total_seconds()
countP = df['Ptime(s)'].notna().sum()
df
#stations[stations['name'] == 'LLW']
#df['sta'].iloc[13]
#countP = df['Ptime(s)'].notna().sum()

sta net     dist      distkm                    time  \
phase                                                       P   
0       CLGH  GB  1.27125  141.356550 2014-03-03 14:29:44.850   
1       GAL1  GB  1.41524  157.367508 2014-03-03 14:29:47.231   
2      IA003  IA  1.19433  132.803437 2014-03-03 14:29:43.734   
3      IA010  IA  0.44838   49.857581 2014-03-03 14:29:29.538   
4      IA012  IA  1.09802  122.094253 2014-03-03 14:29:42.467   
5      IA013  IA  0.88107   97.970514 2014-03-03 14:29:38.985   
6      IAD33  IA  0.64450   71.665130                     NaT   
7       IDGL  EI  1.42932  158.933133 2014-03-03 14:29:48.306   
8       IOMK  GB  1.13340  126.028330 2014-03-03 14:29:43.006   
9       IWEX  EI  1.47335  163.829045 2014-03-03 14:29:47.922   
10     UHEL2  EI  0.57903   64.385198 2014-03-03 14:29:32.917   
11     USUGA  EI  0.65428   72.752617 2014-03-03 14:29:33.801   

                              Ptime(s)  
phase                       S           
0     2014-03-03 14:30:02.513   23.139  
1     2014-03-03 14:30:06.392   25.520  
2     2014-03-03 14:30:00.171   22.023  
3     2014-03-03 14:29:35.260    7.827  
4                         NaT   20.756  
5     2014-03-03 14:29:51.006   17.274  
6     2014-03-03 14:29:42.864      NaN  
7     2014-03-03 14:30:07.160   26.595  
8     2014-03-03 14:29:58.481   21.295  
9     2014-03-03 14:30:07.659   26.211  
10                        NaT   11.206  
11    2014-03-03 14:29:43.709   12.090

In [7]:
#Source line : id_src year month day hour min sec lat lon dep_km magnitude num_recs id_event (weight)
#Currently building it up with only P phases !!!!

# Open the file in write mode to ensure it's empty before starting the loop
with open(seisblast_files + '/seismic_tomography_input_P.txt', 'w') as f:
    f.write("")

for index_src in range(len(relocated_catalogue)):
        
    origin_time = pd.to_datetime((relocated_catalogue.loc[index_src, 'DATE'] + ' ' + relocated_catalogue.loc[index_src, 'TIME']), format='%Y-%m-%d %H:%M:%S.%f')

    # Receiver line : id_src id_rec name_rec lat lon elevation_m phase arrival_time_sec (weight)
    ev_id = relocated_catalogue['EVENT-ID'].iloc[index_src]
    df, info = sb.list_picks(james_catalogue, james_bulletins_dir, ev_id)
    df['Ptime(s)'] = (df['time']['P'] - origin_time).dt.total_seconds()
    countP = df['Ptime(s)'].notna().sum()
    
    with open(seisblast_files + '/seismic_tomography_input_P.txt', 'a') as f:
        f.write(
            f"{index_src:<8}"
            f"{relocated_catalogue['year'].iloc[index_src]:<8}"
            f"{relocated_catalogue['month'].iloc[index_src]:<8}"
            f"{relocated_catalogue['day'].iloc[index_src]:<8}"
            f"{relocated_catalogue['hour'].iloc[index_src]:<8}"
            f"{relocated_catalogue['minute'].iloc[index_src]:<8}"
            f"{relocated_catalogue['second'].iloc[index_src]:<12.2f}"
            f"{relocated_catalogue['lat-quarry'].iloc[index_src]:<12}"
            f"{relocated_catalogue['lon-quarry'].iloc[index_src]:<12}"
            f"{relocated_catalogue['depth-quarry'].iloc[index_src]:<12}"
            f"{relocated_catalogue['MAG(ML)'].iloc[index_src]:<12}"
            f"{countP:<12}"
            f"{relocated_catalogue['EVENT-ID'].iloc[index_src]:<12}\n"
        )
        
        for index_rec in range(len(df)):
            # fetch station information
            station_info = stations[stations['name'] == df['sta'].iloc[index_rec]]
            
            # skip if P is not available 
            if pd.notna(df['time']['P'].iloc[index_rec]):
                if not station_info.empty:
                    
                    # write the receiver lines to file
                    f.write(
                        f"{index_src:<8}"
                        f"{index_rec:<8}"
                        f"{df['sta'].iloc[index_rec]:<10}"
                        f"{station_info['lat'].item():<12}"
                        f"{station_info['lon'].item():<12}"
                        f"{station_info['elevation'].item():<12}"
                        f"{'P':<8}"
                        f"{df['Ptime(s)'].iloc[index_rec]:<12}\n"
                    )
                else:
                    print(f"Station {df['sta'].iloc[index_rec]} not found in the station list")
                    continue
            else:
                continue